# Agent Memory: Building Memory-Enabled Investment Agents with LangGraph

In this notebook, we'll explore **agent memory systems** - the ability for AI agents to remember information across interactions. We'll implement all five memory types from the **CoALA (Cognitive Architectures for Language Agents)** framework while building a Stone Ridge Investment Advisory Assistant.

**Learning Objectives:**
- Understand the 5 memory types from the CoALA framework
- Implement short-term memory with checkpointers and thread_id
- Build long-term memory with InMemoryStore and namespaces
- Use semantic memory for meaning-based retrieval
- Apply episodic memory for few-shot learning from past experiences
- Create procedural memory for self-improving agents
- Combine all memory types into a unified investment advisory agent

## Table of Contents:

- **Breakout Room #1:** Memory Foundations
  - Task 1: Dependencies
  - Task 2: Understanding Agent Memory (CoALA Framework)
  - Task 3: Short-Term Memory (MemorySaver, thread_id)
  - Task 4: Long-Term Memory (InMemoryStore, namespaces)
  - Task 5: Message Trimming & Context Management
  - Question #1 & Question #2
  - 🏗️ Activity #1: Store & Retrieve User Investment Profile

- **Breakout Room #2:** Advanced Memory & Integration
  - Task 6: Semantic Memory (Embeddings + Search)
  - Task 7: Building Semantic Investment Knowledge Base
  - Task 8: Episodic Memory (Few-Shot Learning)
  - Task 9: Procedural Memory (Self-Improving Agent)
  - Task 10: Unified Investment Memory Agent
  - Question #3 & Question #4
  - 🏗️ Activity #2: Investment Memory Dashboard

---
# 🤝 Breakout Room #1
## Memory Foundations

## Task 1: Dependencies

Before we begin, make sure you have:

1. **API Keys** for:
   - OpenAI (for GPT-4o-mini and embeddings)
   - LangSmith (optional, for tracing)

2. **Dependencies installed** via `uv sync`

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

In [2]:
# Set API Keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [3]:
# Optional: LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Agent Memory - Investment - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key (press Enter to skip): ") or ""

if not os.environ["LANGCHAIN_API_KEY"]:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")
else:
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")

LangSmith tracing enabled. Project: AIE9 - Agent Memory - Investment - 340d0701


In [4]:
# Initialize LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Test the connection
response = llm.invoke("Say 'Memory systems ready!' in exactly those words.")
print(response.content)

Memory systems ready!


## Task 2: Understanding Agent Memory (CoALA Framework)

The **CoALA (Cognitive Architectures for Language Agents)** framework identifies 5 types of memory that agents can use:

| Memory Type | Human Analogy | AI Implementation | Investment Example |
|-------------|---------------|-------------------|------------------|
| **Short-term** | What someone just said | Conversation history within a thread | Current investment consultation conversation |
| **Long-term** | Remembering a friend's birthday | User preferences stored across sessions | User's risk tolerance, portfolio size, investment goals |
| **Semantic** | Knowing Paris is in France | Facts retrieved by meaning | Investment knowledge retrieval |
| **Episodic** | Remembering your first day at work | Learning from past experiences | Past successful advisory patterns |
| **Procedural** | Knowing how to ride a bike | Self-improving instructions | Learned communication and advisory preferences |

### Memory Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                LangGraph Investment Advisory Agent               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐           │
│  │  Short-term  │  │  Long-term   │  │   Semantic   │           │
│  │    Memory    │  │    Memory    │  │    Memory    │           │
│  │              │  │              │  │              │           │
│  │ Checkpointer │  │    Store     │  │Store+Embed   │           │
│  │ + thread_id  │  │ + namespace  │  │  + search()  │           │
│  └──────────────┘  └──────────────┘  └──────────────┘           │
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐                             │
│  │   Episodic   │  │  Procedural  │                             │
│  │    Memory    │  │    Memory    │                             │
│  │              │  │              │                             │
│  │  Few-shot    │  │Self-modifying│                             │
│  │  examples    │  │   prompts    │                             │
│  └──────────────┘  └──────────────┘                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key LangGraph Components

| Component | Memory Type | Scope |
|-----------|-------------|-------|
| `MemorySaver` (Checkpointer) | Short-term | Within a single thread |
| `InMemoryStore` | Long-term, Semantic, Episodic, Procedural | Across all threads |
| `thread_id` | Short-term | Identifies unique conversations |
| Namespaces | All store-based | Organizes memories by user/purpose |

**Documentation:**
- [CoALA Paper](https://arxiv.org/abs/2309.02427)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)

## Task 3: Short-Term Memory (MemorySaver, thread_id)

**Short-term memory** maintains context within a single conversation thread. Think of it like your working memory during a phone call - you remember what was said earlier, but once the call ends, those details fade.

In LangGraph, short-term memory is implemented through:
- **Checkpointer**: Saves the graph state at each step
- **thread_id**: Uniquely identifies each conversation

### How It Works

```
Thread 1: "Hi, I'm Alice"          Thread 2: "What's my name?"
     │                                   │
     ▼                                   ▼
┌──────────────┐                   ┌──────────────┐
│ Checkpointer │                   │ Checkpointer │
│  thread_1    │                   │  thread_2    │
│              │                   │              │
│ ["Hi Alice"] │                   │ [empty]      │
└──────────────┘                   └──────────────┘
     │                                   │
     ▼                                   ▼
"Hi Alice!"                        "I don't know your name"
```

In [5]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define the state schema for our graph
# The `add_messages` annotation tells LangGraph how to update the messages list
class State(TypedDict):
    messages: Annotated[list, add_messages]


# Define our investment chatbot node
def investment_chatbot(state: State):
    """Process the conversation and generate an investment-focused response."""
    system_prompt = SystemMessage(content="""You are a friendly Investment Advisory Assistant. 
Help users with questions about Stone Ridge's investment philosophy, market outlook, 
portfolio strategy, and risk management.
Be supportive and remember details the user shares about themselves.""")
    
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the graph
builder = StateGraph(State)
builder.add_node("chatbot", investment_chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

# Compile with a checkpointer for short-term memory
checkpointer = MemorySaver()
investment_graph = builder.compile(checkpointer=checkpointer)

print("Investment chatbot compiled with short-term memory (checkpointing)")

Investment chatbot compiled with short-term memory (checkpointing)


In [6]:
# Test short-term memory within a thread
config = {"configurable": {"thread_id": "investment_thread_1"}}

# First message - introduce ourselves
response = investment_graph.invoke(
    {"messages": [HumanMessage(content="Hi! My name is Alex and I want to understand Stone Ridge's investment approach.")]},
    config
)
print("User: Hi! My name is Alex and I want to understand Stone Ridge's investment approach.")
print(f"Assistant: {response['messages'][-1].content}")
print()

User: Hi! My name is Alex and I want to understand Stone Ridge's investment approach.
Assistant: Hi Alex! It's great to meet you. Stone Ridge has a unique investment philosophy that focuses on a few key principles. They emphasize a long-term, value-oriented approach, often looking for opportunities in areas that may be overlooked by traditional investors. 

Their investment strategy often includes a mix of alternative investments, such as private equity and real estate, alongside more traditional assets. They also utilize a systematic approach to risk management, aiming to protect capital while seeking attractive returns.

Is there a specific aspect of their investment approach that you're particularly interested in, or any specific goals you have in mind for your investments?



In [7]:
# Second message - test if it remembers (same thread)
response = investment_graph.invoke(
    {"messages": [HumanMessage(content="What's my name and what am I interested in learning about?")]},
    config  # Same config = same thread_id
)
print("User: What's my name and what am I interested in learning about?")
print(f"Assistant: {response['messages'][-1].content}")

User: What's my name and what am I interested in learning about?
Assistant: Your name is Alex, and you're interested in learning about Stone Ridge's investment approach. If there's anything specific you'd like to dive deeper into, such as their market outlook, portfolio strategy, or risk management practices, just let me know!


In [8]:
# New thread - it won't remember Alex!
different_config = {"configurable": {"thread_id": "investment_thread_2"}}

response = investment_graph.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    different_config  # Different thread_id = no memory of Alex
)
print("User (NEW thread): What's my name?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent doesn't know our name because this is a new thread!")

User (NEW thread): What's my name?
Assistant: I'm sorry, but I don't have access to your name or any personal information unless you share it with me. How can I assist you today?

Notice: The agent doesn't know our name because this is a new thread!


In [9]:
# Inspect the state of thread 1
state = investment_graph.get_state(config)
print(f"Thread 1 has {len(state.values['messages'])} messages:")
for msg in state.values['messages']:
    role = "User" if isinstance(msg, HumanMessage) else "Assistant"
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

Thread 1 has 4 messages:
  User: Hi! My name is Alex and I want to understand Stone Ridge's investment approach.
  Assistant: Hi Alex! It's great to meet you. Stone Ridge has a unique investment philosophy ...
  User: What's my name and what am I interested in learning about?
  Assistant: Your name is Alex, and you're interested in learning about Stone Ridge's investm...


## Task 4: Long-Term Memory (InMemoryStore, namespaces)

**Long-term memory** stores information across different conversation threads. This is like remembering that your friend prefers tea over coffee - you remember it every time you meet them, regardless of what you're currently discussing.

In LangGraph, long-term memory uses:
- **Store**: A persistent key-value store
- **Namespaces**: Organize memories by user, application, or context

### Key Difference from Short-Term Memory

| Short-Term (Checkpointer) | Long-Term (Store) |
|---------------------------|-------------------|
| Scoped to a single thread | Shared across all threads |
| Automatic (messages) | Explicit (you decide what to store) |
| Conversation history | User preferences, facts, etc. |

In [10]:
from langgraph.store.memory import InMemoryStore

# Create a store for long-term memory
store = InMemoryStore()

# Namespaces organize memories - typically by user_id and category
user_id = "user_alex"
profile_namespace = (user_id, "profile")
preferences_namespace = (user_id, "preferences")

# Store Alex's investment profile
store.put(profile_namespace, "name", {"value": "Alex"})
store.put(profile_namespace, "goals", {"primary": "long-term growth", "secondary": "income generation"})
store.put(profile_namespace, "constraints", {"risk_tolerance": "moderate", "restrictions": ["no tobacco stocks"], "esg_preference": True})
store.put(profile_namespace, "portfolio", {"size": "$500K", "horizon": "20 years", "current_allocation": "60/40 stocks/bonds"})

# Store Alex's preferences
store.put(preferences_namespace, "communication", {"style": "data-driven", "detail_level": "comprehensive"})
store.put(preferences_namespace, "reporting", {"frequency": "quarterly", "preferred_metrics": ["CAGR", "Sharpe ratio", "max drawdown"]})

print("Stored Alex's profile and preferences in long-term memory")

Stored Alex's profile and preferences in long-term memory


In [11]:
# Retrieve specific memories
name = store.get(profile_namespace, "name")
print(f"Name: {name.value}")

goals = store.get(profile_namespace, "goals")
print(f"Goals: {goals.value}")

# List all memories in a namespace
print("\nAll profile items:")
for item in store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Name: {'value': 'Alex'}
Goals: {'primary': 'long-term growth', 'secondary': 'income generation'}

All profile items:
  name: {'value': 'Alex'}
  goals: {'primary': 'long-term growth', 'secondary': 'income generation'}
  constraints: {'risk_tolerance': 'moderate', 'restrictions': ['no tobacco stocks'], 'esg_preference': True}
  portfolio: {'size': '$500K', 'horizon': '20 years', 'current_allocation': '60/40 stocks/bonds'}


In [12]:
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig

# Define state with user_id for personalization
class PersonalizedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def personalized_investment_chatbot(state: PersonalizedState, config: RunnableConfig, *, store: BaseStore):
    """An investment chatbot that uses long-term memory for personalization."""
    user_id = state["user_id"]
    profile_namespace = (user_id, "profile")
    preferences_namespace = (user_id, "preferences")
    
    # Retrieve user profile from long-term memory
    profile_items = list(store.search(profile_namespace))
    pref_items = list(store.search(preferences_namespace))
    
    # Build context from profile
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items])
    pref_text = "\n".join([f"- {p.key}: {p.value}" for p in pref_items])
    
    system_msg = f"""You are an Investment Advisory Assistant. You know the following about this user:

PROFILE:
{profile_text if profile_text else 'No profile stored.'}

PREFERENCES:
{pref_text if pref_text else 'No preferences stored.'}

Use this information to personalize your responses. Be supportive and helpful."""
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the personalized graph
builder2 = StateGraph(PersonalizedState)
builder2.add_node("chatbot", personalized_investment_chatbot)
builder2.add_edge(START, "chatbot")
builder2.add_edge("chatbot", END)

# Compile with BOTH checkpointer (short-term) AND store (long-term)
personalized_graph = builder2.compile(
    checkpointer=MemorySaver(),
    store=store
)

print("Personalized graph compiled with both short-term and long-term memory")

Personalized graph compiled with both short-term and long-term memory


In [13]:
# Test the personalized chatbot - it knows Alex's profile!
config = {"configurable": {"thread_id": "personalized_thread_1"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="What investment strategy would you recommend for me?")],
        "user_id": "user_alex"
    },
    config
)

print("User: What investment strategy would you recommend for me?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent knows about Alex's risk tolerance and portfolio without him mentioning it!")

User: What investment strategy would you recommend for me?
Assistant: Given your profile and investment goals, I recommend a balanced investment strategy that aligns with your objectives of long-term growth and income generation while adhering to your moderate risk tolerance and ESG preferences. Here’s a comprehensive approach:

### 1. **Asset Allocation**
   - **Equities (60%)**: Focus on a diversified mix of growth and dividend-paying stocks. Given your ESG preference, consider investing in:
     - **ESG-focused ETFs or mutual funds**: These funds typically invest in companies with strong environmental, social, and governance practices.
     - **Sustainable sectors**: Look into technology, renewable energy, and healthcare, which are expected to grow significantly over the long term.
   - **Fixed Income (40%)**: To generate income and reduce volatility, consider:
     - **ESG-compliant bonds**: These can include green bonds or social bonds that fund projects with positive environmenta

In [14]:
# Even in a NEW thread, it still knows Alex's profile
# because long-term memory is cross-thread!

new_config = {"configurable": {"thread_id": "personalized_thread_2"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="Are there any risks I should be aware of given my portfolio?")],
        "user_id": "user_alex"
    },
    new_config
)

print("User (NEW thread): Are there any risks I should be aware of given my portfolio?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: Even in a new thread, the agent knows Alex's portfolio and constraints!")

User (NEW thread): Are there any risks I should be aware of given my portfolio?
Assistant: Given your portfolio's allocation of 60% stocks and 40% bonds, there are several risks to consider, especially in the context of your long-term growth and income generation goals, as well as your moderate risk tolerance:

1. **Market Risk**: With a significant portion of your portfolio in stocks, you are exposed to market volatility. Economic downturns can lead to declines in stock prices, which may affect your long-term growth potential.

2. **Interest Rate Risk**: Your bond allocation is subject to interest rate risk. If interest rates rise, the value of existing bonds may decline, which could impact your portfolio's overall performance, especially if you hold long-term bonds.

3. **Inflation Risk**: Over a 20-year horizon, inflation can erode the purchasing power of your returns. While stocks generally provide a hedge against inflation over the long term, it's important to ensure that your bon

## Task 5: Message Trimming & Context Management

Long conversations can exceed the LLM's context window. LangGraph provides utilities to manage message history:

- **`trim_messages`**: Keeps only recent messages up to a token limit
- **Summarization**: Compress older messages into summaries

### Why Trim Even with 128K Context?

Even with large context windows:
1. **Cost**: More tokens = higher API costs
2. **Latency**: Larger contexts take longer to process
3. **Quality**: Models can struggle with "lost in the middle" - important info buried in long contexts
4. **Relevance**: Old messages may not be relevant to current query

In [15]:
from langchain_core.messages import trim_messages

# Create a trimmer that keeps only recent messages
trimmer = trim_messages(
    max_tokens=500,  # Keep messages up to this token count
    strategy="last",  # Keep the most recent messages
    token_counter=llm,  # Use the LLM to count tokens
    include_system=True,  # Always keep system messages
    allow_partial=False,  # Don't cut messages in half
)

# Example: Create a long conversation
long_conversation = [
    SystemMessage(content="You are an investment advisory assistant."),
    HumanMessage(content="I want to improve my portfolio returns."),
    AIMessage(content="Great goal! Let's start with your current allocation. What does your portfolio look like?"),
    HumanMessage(content="I have about 60% stocks and 40% bonds."),
    AIMessage(content="That's a balanced allocation. For higher returns, you might consider increasing equity exposure or adding alternative investments."),
    HumanMessage(content="What about international diversification?"),
    AIMessage(content="International exposure can reduce risk through diversification. Consider allocating 20-30% to international developed and emerging markets."),
    HumanMessage(content="And alternative investments?"),
    AIMessage(content="Alternatives like reinsurance, real estate, and commodities can provide uncorrelated returns and enhance portfolio efficiency."),
    HumanMessage(content="What's the most important change I should make first?"),
]

# Trim to fit context window
trimmed = trimmer.invoke(long_conversation)
print(f"Original: {len(long_conversation)} messages")
print(f"Trimmed: {len(trimmed)} messages")
print("\nTrimmed conversation:")
for msg in trimmed:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
    print(f"  {role}: {content}")

Original: 10 messages
Trimmed: 10 messages

Trimmed conversation:
  System: You are an investment advisory assistant.
  Human: I want to improve my portfolio returns.
  AI: Great goal! Let's start with your current allocation. What d...
  Human: I have about 60% stocks and 40% bonds.
  AI: That's a balanced allocation. For higher returns, you might ...
  Human: What about international diversification?
  AI: International exposure can reduce risk through diversificati...
  Human: And alternative investments?
  AI: Alternatives like reinsurance, real estate, and commodities ...
  Human: What's the most important change I should make first?


In [16]:
# Summarization approach for longer conversations

def summarize_conversation(messages: list, max_messages: int = 6) -> list:
    """Summarize older messages to manage context length."""
    if len(messages) <= max_messages:
        return messages
    
    # Keep the system message and last few messages
    system_msg = messages[0] if isinstance(messages[0], SystemMessage) else None
    content_messages = messages[1:] if system_msg else messages
    
    if len(content_messages) <= max_messages:
        return messages
    
    old_messages = content_messages[:-max_messages+1]
    recent_messages = content_messages[-max_messages+1:]
    
    # Summarize old messages
    summary_prompt = f"""Summarize this conversation in 2-3 sentences, 
capturing key investment topics discussed and any important user information:

{chr(10).join([f'{type(m).__name__}: {m.content[:200]}' for m in old_messages])}"""
    
    summary = llm.invoke(summary_prompt)
    
    # Return: system + summary + recent messages
    result = []
    if system_msg:
        result.append(system_msg)
    result.append(SystemMessage(content=f"[Previous conversation summary: {summary.content}]"))
    result.extend(recent_messages)
    
    return result


# Test summarization
summarized = summarize_conversation(long_conversation, max_messages=4)
print(f"Summarized: {len(summarized)} messages")
print("\nSummarized conversation:")
for msg in summarized:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

Summarized: 5 messages

Summarized conversation:
  System: You are an investment advisory assistant.
  System: [Previous conversation summary: The user aims to improve their portfolio returns...
  Human: And alternative investments?
  AI: Alternatives like reinsurance, real estate, and commodities can provide uncorrel...
  Human: What's the most important change I should make first?


---
## ❓ Question #1:

What are the trade-offs between **short-term memory** (checkpointer) vs **long-term memory** (store)? When should investment data move from short-term to long-term? Consider:
- What information should persist across sessions?
- What are the compliance implications?
- How would you decide what to promote from short-term to long-term?

##### Answer:
Short-term memory retains factual context only for the duration of conversation thread, whereas long-term memory stores pieces of factual context across threads and over a long period of time. Short-term memory is automatically retained whereas long-term memory has to be explicitly augmented and maintained. In the realm of investment data, long-term memory is necessary for storing compliance information (disclosures, legal limits, who asked what and when, etc.), firm-level risk tolerance data, user-level and system-level preferences and business logic. If a piece of info can be used by multiple users across sessions, or if its required for regulatory reasons than it should be moved from short-term to long-term storage.

## ❓ Question #2:

Why use message trimming with a 128K context window when the Stone Ridge investor letter is relatively small? What should **always** be preserved when trimming an investment consultation?

Consider:
- The "lost in the middle" phenomenon
- Cost and latency implications
- What user information is critical for safety (risk tolerance, constraints, etc.)

##### Answer:
Message trimming helps to keep token use cost down and makes scanning existing context more efficient. If the same body of context is going to be retrieved when answering every question, it's helpful to have that context be as small and as quickly parsable as possible. Any information deemed critical for safety and compliance, or information that will be relevant for future questions should always be retained. Long blobs of context should be periodically summarized to reduce the number of tokens required to scan and integrate that context into future question responses. In summarizing and re-compacting the context, it is important to bubble-up important data that may have otherwise been buried in long blobs of context.

---
## 🏗️ Activity #1: Store & Retrieve User Investment Profile

Build a complete investment profile system that:
1. Defines an investment profile schema (name, risk tolerance, portfolio size, investment horizon, restrictions, goals)
2. Creates functions to store and retrieve profile data
3. Builds a personalized investment agent that uses the profile
4. Tests that different users get different advice

### Requirements:
- Define at least 5 profile attributes
- Support multiple users with different profiles
- Agent should reference profile data in responses

In [17]:
### YOUR CODE HERE ###

# Step 1: Define an investment profile schema
PROFILE_SCHEMA = {
    "name": str,
    "risk_tolerance": str,       # conservative, moderate, aggressive
    "portfolio_size": str,       # e.g. "$500K"
    "investment_horizon": str,   # e.g. "20 years"
    "restrictions": list,        # e.g. ["no tobacco stocks"]
    "goals": dict,               # e.g. {"primary": "growth", "secondary": "income"}
    "preferred_asset_classes": list,  # e.g. ["equities", "alternatives"]
    "esg_preference": bool,
}


# Step 2: Create helper functions to store and retrieve profiles
def store_investment_profile(store, user_id: str, profile: dict):
    """Store a user's investment profile in long-term memory."""
    namespace = (user_id, "profile")
    for key, value in profile.items():
        store.put(namespace, key, {"value": value})
    print(f"Stored profile for {user_id} with {len(profile)} attributes")


def get_investment_profile(store, user_id: str) -> dict:
    """Retrieve a user's investment profile from long-term memory."""
    namespace = (user_id, "profile")
    items = list(store.search(namespace))
    return {item.key: item.value["value"] for item in items}


# Step 3: Create two different user profiles
profile_conservative = {
    "name": "Sarah",
    "risk_tolerance": "conservative",
    "portfolio_size": "$1.2M",
    "investment_horizon": "5 years",
    "restrictions": ["no crypto", "no leveraged ETFs"],
    "goals": {"primary": "capital preservation", "secondary": "steady income"},
    "preferred_asset_classes": ["investment-grade bonds", "dividend stocks", "cash equivalents"],
    "esg_preference": False,
}

profile_aggressive = {
    "name": "Marcus",
    "risk_tolerance": "aggressive",
    "portfolio_size": "$200K",
    "investment_horizon": "30 years",
    "restrictions": [],
    "goals": {"primary": "maximum long-term growth", "secondary": "compounding"},
    "preferred_asset_classes": ["growth equities", "alternatives", "emerging markets", "crypto"],
    "esg_preference": True,
}

# Use the existing store from earlier cells
store_investment_profile(store, "user_sarah", profile_conservative)
store_investment_profile(store, "user_marcus", profile_aggressive)

# Verify retrieval
print("\nSarah's profile:", get_investment_profile(store, "user_sarah"))
print("\nMarcus's profile:", get_investment_profile(store, "user_marcus"))


# Step 4: Build a personalized agent that uses profiles
def profile_aware_chatbot(state: PersonalizedState, config: RunnableConfig, *, store: BaseStore):
    """Investment chatbot that personalizes advice based on stored user profile."""
    user_id = state["user_id"]
    profile = get_investment_profile(store, user_id)

    if profile:
        profile_text = "\n".join([f"- {k}: {v}" for k, v in profile.items()])
        system_msg = f"""You are an Investment Advisory Assistant. Personalize your advice based on this client profile:

{profile_text}

Tailor recommendations to their risk tolerance, goals, horizon, and restrictions.
Reference specific profile details in your response."""
    else:
        system_msg = "You are an Investment Advisory Assistant. Ask the user about their investment profile."

    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


profile_builder = StateGraph(PersonalizedState)
profile_builder.add_node("chatbot", profile_aware_chatbot)
profile_builder.add_edge(START, "chatbot")
profile_builder.add_edge("chatbot", END)

profile_graph = profile_builder.compile(
    checkpointer=MemorySaver(),
    store=store
)
print("\nProfile-aware agent compiled")


# Step 5: Test with different users - they should get different advice
question = "What asset allocation would you recommend for me?"

print("\n" + "=" * 60)
print("SARAH (conservative, 5-year horizon, $1.2M)")
print("=" * 60)
response = profile_graph.invoke(
    {"messages": [HumanMessage(content=question)], "user_id": "user_sarah"},
    {"configurable": {"thread_id": "profile_test_sarah"}}
)
print(f"Assistant: {response['messages'][-1].content}")

print("\n" + "=" * 60)
print("MARCUS (aggressive, 30-year horizon, $200K)")
print("=" * 60)
response = profile_graph.invoke(
    {"messages": [HumanMessage(content=question)], "user_id": "user_marcus"},
    {"configurable": {"thread_id": "profile_test_marcus"}}
)
print(f"Assistant: {response['messages'][-1].content}")

Stored profile for user_sarah with 8 attributes
Stored profile for user_marcus with 8 attributes

Sarah's profile: {'name': 'Sarah', 'risk_tolerance': 'conservative', 'portfolio_size': '$1.2M', 'investment_horizon': '5 years', 'restrictions': ['no crypto', 'no leveraged ETFs'], 'goals': {'primary': 'capital preservation', 'secondary': 'steady income'}, 'preferred_asset_classes': ['investment-grade bonds', 'dividend stocks', 'cash equivalents'], 'esg_preference': False}

Marcus's profile: {'name': 'Marcus', 'risk_tolerance': 'aggressive', 'portfolio_size': '$200K', 'investment_horizon': '30 years', 'restrictions': [], 'goals': {'primary': 'maximum long-term growth', 'secondary': 'compounding'}, 'preferred_asset_classes': ['growth equities', 'alternatives', 'emerging markets', 'crypto'], 'esg_preference': True}

Profile-aware agent compiled

SARAH (conservative, 5-year horizon, $1.2M)
Assistant: Based on your profile, Sarah, here’s a tailored asset allocation strategy that aligns with yo

---
# 🤝 Breakout Room #2
## Advanced Memory & Integration

## Task 6: Semantic Memory (Embeddings + Search)

**Semantic memory** stores facts and retrieves them based on *meaning* rather than exact matches. This is like how you might remember "that fund with the great risk-adjusted returns" even if you can't remember its exact name.

In LangGraph, semantic memory uses:
- **Store with embeddings**: Converts text to vectors for similarity search
- **`store.search()`**: Finds relevant memories by semantic similarity

### How It Works

```
User asks: "What helps with portfolio diversification?"
         ↓
Query embedded → [0.2, 0.8, 0.1, ...]
         ↓
Compare with stored investment facts:
  - "Uncorrelated assets reduce portfolio risk" → 0.92 similarity ✓
  - "Rebalancing maintains target allocations" → 0.35 similarity
         ↓
Return: "Uncorrelated assets reduce portfolio risk"
```

In [18]:
from langchain_openai import OpenAIEmbeddings

# Create embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create a store with semantic search enabled
semantic_store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,  # Dimension of text-embedding-3-small
    }
)

print("Semantic memory store created with embedding support")

Semantic memory store created with embedding support


In [19]:
# Store various investment facts as semantic memories
namespace = ("investment", "facts")

investment_facts = [
    ("fact_1", {"text": "Diversification across uncorrelated assets can reduce portfolio risk without sacrificing returns"}),
    ("fact_2", {"text": "Stone Ridge focuses on alternative risk premiums including reinsurance and longevity risk"}),
    ("fact_3", {"text": "Tail risk hedging provides insurance against extreme market downturns"}),
    ("fact_4", {"text": "A long-term investment horizon allows investors to capture illiquidity premiums"}),
    ("fact_5", {"text": "Factor investing targets specific drivers of return such as value, momentum, and quality"}),
    ("fact_6", {"text": "Rebalancing portfolios periodically helps maintain target risk levels"}),
    ("fact_7", {"text": "Alternative investments like reinsurance have low correlation with traditional stock and bond markets"}),
    ("fact_8", {"text": "Systematic risk management frameworks help identify and mitigate portfolio vulnerabilities"}),
]

for key, value in investment_facts:
    semantic_store.put(namespace, key, value)

print(f"Stored {len(investment_facts)} investment facts in semantic memory")

Stored 8 investment facts in semantic memory


In [20]:
# Search semantically - notice we don't need exact matches!

queries = [
    "How can I protect my portfolio from a market crash?",
    "What alternative investments should I consider?",
    "How should I think about risk in my portfolio?",
    "What is Stone Ridge's investment approach?",
]

for query in queries:
    print(f"\nQuery: {query}")
    results = semantic_store.search(namespace, query=query, limit=2)
    for r in results:
        print(f"   {r.value['text']} (score: {r.score:.3f})")


Query: How can I protect my portfolio from a market crash?
   Tail risk hedging provides insurance against extreme market downturns (score: 0.454)
   Rebalancing portfolios periodically helps maintain target risk levels (score: 0.442)

Query: What alternative investments should I consider?
   Alternative investments like reinsurance have low correlation with traditional stock and bond markets (score: 0.568)
   Diversification across uncorrelated assets can reduce portfolio risk without sacrificing returns (score: 0.407)

Query: How should I think about risk in my portfolio?
   Rebalancing portfolios periodically helps maintain target risk levels (score: 0.493)
   Systematic risk management frameworks help identify and mitigate portfolio vulnerabilities (score: 0.471)

Query: What is Stone Ridge's investment approach?
   Stone Ridge focuses on alternative risk premiums including reinsurance and longevity risk (score: 0.641)
   Factor investing targets specific drivers of return such as

## Task 7: Building Semantic Investment Knowledge Base

Let's load the Stone Ridge 2025 Investor Letter and create a semantic knowledge base that our agent can search.

This is similar to RAG from Module 4, but now using LangGraph's Store API instead of a separate vector database.

In [21]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load and chunk the investment document
loader = PyMuPDFLoader("data/Stone Ridge 2025 Investor Letter.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)

print(f"Loaded and split into {len(chunks)} chunks")
print(f"\nSample chunk:\n{chunks[0].page_content[:200]}...")

Loaded and split into 127 chunks

Sample chunk:
2025 Investor Letter...


In [22]:
# Store chunks in semantic memory
knowledge_namespace = ("investment", "knowledge")

for i, chunk in enumerate(chunks):
    semantic_store.put(
        knowledge_namespace,
        f"chunk_{i}",
        {"text": chunk.page_content, "source": "Stone Ridge 2025 Investor Letter.pdf"}
    )

print(f"Stored {len(chunks)} chunks in semantic knowledge base")

Stored 127 chunks in semantic knowledge base


In [23]:
# Build a semantic search investment chatbot

class SemanticState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def semantic_investment_chatbot(state: SemanticState, config: RunnableConfig, *, store: BaseStore):
    """An investment chatbot that retrieves relevant facts using semantic search."""
    user_message = state["messages"][-1].content
    
    # Search for relevant knowledge
    knowledge_results = store.search(
        ("investment", "knowledge"),
        query=user_message,
        limit=3
    )
    
    # Build context from retrieved knowledge
    if knowledge_results:
        knowledge_text = "\n\n".join([f"- {r.value['text']}" for r in knowledge_results])
        system_msg = f"""You are an Investment Advisory Assistant with access to the Stone Ridge investor letter knowledge base.

Relevant information from your knowledge base:
{knowledge_text}

Use this information to answer the user's question. If the information doesn't directly answer their question, use your general knowledge but mention what you found."""
    else:
        system_msg = "You are an Investment Advisory Assistant. Answer investment questions helpfully."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build and compile
builder3 = StateGraph(SemanticState)
builder3.add_node("chatbot", semantic_investment_chatbot)
builder3.add_edge(START, "chatbot")
builder3.add_edge("chatbot", END)

semantic_graph = builder3.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Semantic investment chatbot ready")

Semantic investment chatbot ready


In [24]:
# Test semantic retrieval
config = {"configurable": {"thread_id": "semantic_thread_1"}}

questions = [
    "What is Stone Ridge's view on the current market environment?",
    "How does Stone Ridge approach risk management?",
    "What is Stone Ridge's investment philosophy?",
]

for q in questions:
    response = semantic_graph.invoke(
        {"messages": [HumanMessage(content=q)], "user_id": "test_user"},
        config
    )
    print(f"\nUser: {q}")
    print(f"Assistant: {response['messages'][-1].content[:500]}...")


User: What is Stone Ridge's view on the current market environment?
Assistant: The information I have access to does not provide specific insights or views from Stone Ridge regarding the current market environment. It is important to note that any opinions or analyses expressed in their communications do not represent a formal or official view of the firm and should not be interpreted as investment recommendations. 

For the most accurate and up-to-date perspectives from Stone Ridge, I would recommend checking their latest investor letters or official communications direct...

User: How does Stone Ridge approach risk management?
Assistant: While the specific details on Stone Ridge's approach to risk management are not provided in the information I have access to, it is generally understood that effective risk management is a critical component of any investment strategy. Stone Ridge emphasizes a disciplined operating model and a focus on building products that they would want for them

## Task 8: Episodic Memory (Few-Shot Learning)

**Episodic memory** stores past experiences and uses them as examples for future tasks. In humans, this is like remembering the first time you provided a successful investment recommendation and using that experience to guide future interactions.

For AI agents, episodic memory is often implemented as **few-shot examples** - showing the model examples of correct behavior so it can learn patterns.

### The CoALA Framework Perspective

> "Facts can be written to semantic memory, whereas **experiences** can be written to episodic memory."

Key insight: Sometimes it's easier to **show** than **tell**. LLMs learn well from examples!

In [25]:
# Create episodic memory with past successful interactions
episodic_namespace = ("agent", "episodes")

successful_episodes = [
    {
        "situation": "User asked about portfolio diversification with concentrated holdings",
        "input": "I have most of my portfolio in tech stocks. How should I diversify?",
        "output": """For a concentrated tech portfolio, I'd recommend diversifying across several dimensions:

1. **Asset classes** - Add bonds, real estate (REITs), and alternatives
2. **Geographies** - Include international developed and emerging markets
3. **Sectors** - Shift allocation toward healthcare, consumer staples, and industrials
4. **Alternative risk premiums** - Consider reinsurance or longevity-linked investments

Start by gradually rebalancing 5-10% per quarter to avoid tax impacts. Would you like a specific rebalancing plan?""",
        "feedback": "User appreciated the structured approach and specific action items"
    },
    {
        "situation": "User concerned about market downturn impact on retirement portfolio",
        "input": "I'm 10 years from retirement and worried about a market crash. What should I do?",
        "output": """Market timing is difficult, but there are proven strategies to protect near-retirement portfolios:

**Risk reduction:**
- Gradually shift to a more conservative allocation (e.g., 50/50 stocks/bonds)
- Consider tail risk hedging for catastrophic downside protection

**Income stability:**
- Build a 2-3 year cash buffer for living expenses
- Add dividend-paying stocks and investment-grade bonds

**Stay invested:**
- Keep some equity exposure for growth to combat inflation
- Focus on quality companies with strong balance sheets

Which aspect would you like to explore in more detail?""",
        "feedback": "User found the balanced approach reassuring and actionable"
    },
    {
        "situation": "User asking about alternative investments",
        "input": "What are alternative investments and should I have them in my portfolio?",
        "output": """Alternative investments are assets beyond traditional stocks and bonds. They can add diversification and unique return sources:

**Common alternatives:**
- Real estate (REITs, direct property)
- Private equity and venture capital
- Hedge funds and managed futures
- Reinsurance and catastrophe bonds
- Commodities and natural resources

**Benefits:**
- Low correlation with traditional markets
- Access to different risk premiums
- Potential for higher risk-adjusted returns

**Considerations:**
- Often less liquid than public markets
- May require higher minimum investments
- Due diligence is critical

A typical allocation might be 10-20% of your portfolio. What's your current portfolio size and investment horizon?""",
        "feedback": "User valued the comprehensive overview without being overwhelming"
    },
]

for i, episode in enumerate(successful_episodes):
    semantic_store.put(
        episodic_namespace,
        f"episode_{i}",
        {
            "text": episode["situation"],  # Used for semantic search
            **episode
        }
    )

print(f"Stored {len(successful_episodes)} episodic memories (past successful interactions)")

Stored 3 episodic memories (past successful interactions)


In [26]:
class EpisodicState(TypedDict):
    messages: Annotated[list, add_messages]


def episodic_investment_chatbot(state: EpisodicState, config: RunnableConfig, *, store: BaseStore):
    """A chatbot that learns from past successful interactions."""
    user_question = state["messages"][-1].content
    
    # Search for similar past experiences
    similar_episodes = store.search(
        ("agent", "episodes"),
        query=user_question,
        limit=1
    )
    
    # Build few-shot examples from past episodes
    if similar_episodes:
        episode = similar_episodes[0].value
        few_shot_example = f"""Here's an example of a similar investment question I handled well:

User asked: {episode['input']}

My response was:
{episode['output']}

The user feedback was: {episode['feedback']}

Use this as inspiration for the style, structure, and tone of your response, but tailor it to the current question."""
        
        system_msg = f"""You are an Investment Advisory Assistant. Learn from your past successes:

{few_shot_example}"""
    else:
        system_msg = "You are an Investment Advisory Assistant. Be helpful, specific, and supportive."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the episodic memory graph
builder4 = StateGraph(EpisodicState)
builder4.add_node("chatbot", episodic_investment_chatbot)
builder4.add_edge(START, "chatbot")
builder4.add_edge("chatbot", END)

episodic_graph = builder4.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Episodic memory chatbot ready")

Episodic memory chatbot ready


In [27]:
# Test episodic memory - similar question to stored episode
config = {"configurable": {"thread_id": "episodic_thread_1"}}

response = episodic_graph.invoke(
    {"messages": [HumanMessage(content="I'm thinking about adding some alternative investments to my portfolio. What should I consider?")]},
    config
)

print("User: I'm thinking about adding some alternative investments to my portfolio. What should I consider?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The response structure mirrors the successful alternatives episode!")

User: I'm thinking about adding some alternative investments to my portfolio. What should I consider?

Assistant: Adding alternative investments to your portfolio can be a strategic move to enhance diversification and potentially improve returns. Here are some key considerations to keep in mind:

**Types of Alternative Investments:**
- **Real Estate:** This includes Real Estate Investment Trusts (REITs) or direct property ownership, which can provide rental income and capital appreciation.
- **Private Equity:** Investing in private companies or funds that target private equity can offer high returns, though they often come with higher risk and longer investment horizons.
- **Hedge Funds:** These funds employ various strategies to generate returns, including long/short equity, market neutral, and global macro strategies.
- **Commodities:** Investing in physical goods like gold, oil, or agricultural products can hedge against inflation and provide diversification.
- **Cryptocurrencies:**

## Task 9: Procedural Memory (Self-Improving Agent)

**Procedural memory** stores the rules and instructions that guide behavior. In humans, this is like knowing *how* to give good advice - it's internalized knowledge about performing tasks.

For AI agents, procedural memory often means **self-modifying prompts**. The agent can:
1. Store its current instructions in the memory store
2. Reflect on feedback from interactions
3. Update its own instructions to improve

### The Reflection Pattern

```
User feedback: "Your advice is too long and complicated"
         ↓
Agent reflects on current instructions
         ↓
Agent updates instructions: "Keep advice concise and actionable"
         ↓
Future responses use updated instructions
```

In [28]:
# Initialize procedural memory with base instructions
procedural_namespace = ("agent", "instructions")

initial_instructions = """You are an Investment Advisory Assistant.

Guidelines:
- Be objective and data-driven in your analysis
- Provide evidence-based investment information
- Ask clarifying questions about risk tolerance and investment goals
- Present balanced perspectives on investment decisions
- Always note that past performance doesn't guarantee future results"""

semantic_store.put(
    procedural_namespace,
    "investment_assistant",
    {"instructions": initial_instructions, "version": 1}
)

print("Initialized procedural memory with base instructions")
print(f"\nCurrent Instructions (v1):\n{initial_instructions}")

Initialized procedural memory with base instructions

Current Instructions (v1):
You are an Investment Advisory Assistant.

Guidelines:
- Be objective and data-driven in your analysis
- Provide evidence-based investment information
- Ask clarifying questions about risk tolerance and investment goals
- Present balanced perspectives on investment decisions
- Always note that past performance doesn't guarantee future results


In [29]:
class ProceduralState(TypedDict):
    messages: Annotated[list, add_messages]
    feedback: str  # Optional feedback from user


def get_instructions(store: BaseStore) -> tuple[str, int]:
    """Retrieve current instructions from procedural memory."""
    item = store.get(("agent", "instructions"), "investment_assistant")
    if item is None:
        return "You are a helpful investment advisory assistant.", 0
    return item.value["instructions"], item.value["version"]


def procedural_assistant_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Respond using current procedural instructions."""
    instructions, version = get_instructions(store)
    
    messages = [SystemMessage(content=instructions)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


def reflection_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Reflect on feedback and update instructions if needed."""
    feedback = state.get("feedback", "")
    
    if not feedback:
        return {}  # No feedback, no update needed
    
    # Get current instructions
    current_instructions, version = get_instructions(store)
    
    # Ask the LLM to reflect and improve instructions
    reflection_prompt = f"""You are improving an investment advisory assistant's instructions based on user feedback.

Current Instructions:
{current_instructions}

User Feedback:
{feedback}

Based on this feedback, provide improved instructions. Keep the same general format but incorporate the feedback.
Only output the new instructions, nothing else."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    new_instructions = response.content
    
    # Update procedural memory with new instructions
    store.put(
        ("agent", "instructions"),
        "investment_assistant",
        {"instructions": new_instructions, "version": version + 1}
    )
    
    print(f"\nInstructions updated to version {version + 1}")
    return {}


def should_reflect(state: ProceduralState) -> str:
    """Decide whether to reflect on feedback."""
    if state.get("feedback"):
        return "reflect"
    return "end"


# Build the procedural memory graph
builder5 = StateGraph(ProceduralState)
builder5.add_node("assistant", procedural_assistant_node)
builder5.add_node("reflect", reflection_node)

builder5.add_edge(START, "assistant")
builder5.add_conditional_edges("assistant", should_reflect, {"reflect": "reflect", "end": END})
builder5.add_edge("reflect", END)

procedural_graph = builder5.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Procedural memory graph ready (with self-improvement capability)")

Procedural memory graph ready (with self-improvement capability)


In [30]:
# Test with initial instructions
config = {"configurable": {"thread_id": "procedural_thread_1"}}

response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How should I think about portfolio risk?")],
        "feedback": ""  # No feedback yet
    },
    config
)

print("User: How should I think about portfolio risk?")
print(f"\nAssistant (v1 instructions):\n{response['messages'][-1].content}")

User: How should I think about portfolio risk?

Assistant (v1 instructions):
Thinking about portfolio risk involves understanding several key concepts and factors that can influence the performance of your investments. Here are some important considerations:

1. **Risk Tolerance**: Assess your personal risk tolerance, which is your ability and willingness to endure fluctuations in the value of your investments. This can be influenced by factors such as your investment goals, time horizon, and financial situation. Would you consider yourself conservative, moderate, or aggressive in your investment approach?

2. **Diversification**: Diversifying your portfolio across different asset classes (stocks, bonds, real estate, etc.) and sectors can help mitigate risk. By spreading investments, you reduce the impact of a poor-performing asset on your overall portfolio. Have you considered how diversified your current portfolio is?

3. **Volatility**: Understand that different asset classes have v

In [31]:
# Now provide feedback - the agent will update its own instructions!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How should I think about portfolio risk?")],
        "feedback": "Your responses are too long. Please be more concise and give me 3 actionable insights maximum."
    },
    {"configurable": {"thread_id": "procedural_thread_2"}}
)


Instructions updated to version 2


In [32]:
# Check the updated instructions
new_instructions, version = get_instructions(semantic_store)
print(f"Updated Instructions (v{version}):\n")
print(new_instructions)

Updated Instructions (v2):

You are an Investment Advisory Assistant.

Guidelines:
- Be objective and data-driven in your analysis.
- Provide up to 3 actionable insights based on evidence-based investment information.
- Ask clarifying questions about risk tolerance and investment goals.
- Present balanced perspectives on investment decisions.
- Always note that past performance doesn't guarantee future results.


In [33]:
# Test with updated instructions - should be more concise now!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="What investment opportunities should I consider in the current market?")],
        "feedback": ""  # No feedback this time
    },
    {"configurable": {"thread_id": "procedural_thread_3"}}
)

print(f"User: What investment opportunities should I consider in the current market?")
print(f"\nAssistant (v{version} instructions - after feedback):")
print(response['messages'][-1].content)
print("\nNotice: The response should now be more concise based on the feedback!")

User: What investment opportunities should I consider in the current market?

Assistant (v2 instructions - after feedback):
To provide tailored investment opportunities, I would need to understand your risk tolerance and investment goals. Here are a few clarifying questions:

1. What is your risk tolerance? Are you more conservative, moderate, or aggressive in your investment approach?
2. What are your investment goals? Are you looking for long-term growth, income generation, or capital preservation?
3. What is your investment horizon? Are you looking to invest for the short term (1-3 years), medium term (3-5 years), or long term (5+ years)?

In the current market, here are three actionable insights based on recent trends and data:

1. **Diversified ETFs**: Exchange-Traded Funds (ETFs) that focus on sectors like technology, healthcare, or renewable energy have shown resilience and growth potential. Consider ETFs that track indices with strong fundamentals and growth prospects. However,

## Task 10: Unified Investment Memory Agent

Now let's combine **all 5 memory types** into a unified investment advisory agent:

1. **Short-term**: Remembers current conversation (checkpointer)
2. **Long-term**: Stores user profile across sessions (store + namespace)
3. **Semantic**: Retrieves relevant investment knowledge (store + embeddings)
4. **Episodic**: Uses past successful interactions as examples (store + search)
5. **Procedural**: Adapts behavior based on feedback (store + reflection)

### Memory Retrieval Flow

```
User Query: "What investment strategy suits my risk profile?"
              │
              ▼
┌─────────────────────────────────────────────────┐
│  1. PROCEDURAL: Get current instructions         │
│  2. LONG-TERM: Load user profile (constraints)   │
│  3. SEMANTIC: Search investment knowledge        │
│  4. EPISODIC: Find similar past interactions     │
│  5. SHORT-TERM: Include conversation history     │
└─────────────────────────────────────────────────┘
              │
              ▼
        Generate personalized, informed response
```

In [34]:
class UnifiedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str
    feedback: str


def unified_investment_assistant(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """An assistant that uses all five memory types."""
    user_id = state["user_id"]
    user_message = state["messages"][-1].content
    
    # 1. PROCEDURAL: Get current instructions
    instructions_item = store.get(("agent", "instructions"), "investment_assistant")
    base_instructions = instructions_item.value["instructions"] if instructions_item else "You are a helpful investment advisory assistant."
    
    # 2. LONG-TERM: Get user profile
    profile_items = list(store.search((user_id, "profile")))
    pref_items = list(store.search((user_id, "preferences")))
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items]) if profile_items else "No profile stored."
    
    # 3. SEMANTIC: Search for relevant knowledge
    relevant_knowledge = store.search(("investment", "knowledge"), query=user_message, limit=2)
    knowledge_text = "\n".join([f"- {r.value['text'][:200]}..." for r in relevant_knowledge]) if relevant_knowledge else "No specific knowledge found."
    
    # 4. EPISODIC: Find similar past interactions
    similar_episodes = store.search(("agent", "episodes"), query=user_message, limit=1)
    if similar_episodes:
        ep = similar_episodes[0].value
        episode_text = f"Similar past interaction:\nUser: {ep.get('input', 'N/A')}\nResponse style: {ep.get('feedback', 'N/A')}"
    else:
        episode_text = "No similar past interactions found."
    
    # Build comprehensive system message
    system_message = f"""{base_instructions}

=== USER PROFILE ===
{profile_text}

=== RELEVANT INVESTMENT KNOWLEDGE ===
{knowledge_text}

=== LEARNING FROM EXPERIENCE ===
{episode_text}

Use all of this context to provide the best possible personalized response."""
    
    # 5. SHORT-TERM: Full conversation history is automatically managed by the checkpointer
    # Use summarization for long conversations
    trimmed_messages = summarize_conversation(state["messages"], max_messages=6)
    
    messages = [SystemMessage(content=system_message)] + trimmed_messages
    response = llm.invoke(messages)
    return {"messages": [response]}


def unified_feedback_node(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """Update procedural memory based on feedback."""
    feedback = state.get("feedback", "")
    if not feedback:
        return {}
    
    item = store.get(("agent", "instructions"), "investment_assistant")
    if item is None:
        return {}
    
    current = item.value
    reflection_prompt = f"""Update these instructions based on feedback:

Current: {current['instructions']}
Feedback: {feedback}

Output only the updated instructions."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    store.put(
        ("agent", "instructions"),
        "investment_assistant",
        {"instructions": response.content, "version": current["version"] + 1}
    )
    print(f"Procedural memory updated to v{current['version'] + 1}")
    return {}


def unified_route(state: UnifiedState) -> str:
    return "feedback" if state.get("feedback") else "end"


# Build the unified graph
unified_builder = StateGraph(UnifiedState)
unified_builder.add_node("assistant", unified_investment_assistant)
unified_builder.add_node("feedback", unified_feedback_node)

unified_builder.add_edge(START, "assistant")
unified_builder.add_conditional_edges("assistant", unified_route, {"feedback": "feedback", "end": END})
unified_builder.add_edge("feedback", END)

# Compile with both checkpointer (short-term) and store (all other memory types)
unified_graph = unified_builder.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Unified investment assistant ready with all 5 memory types!")

Unified investment assistant ready with all 5 memory types!


In [35]:
# Test the unified assistant
config = {"configurable": {"thread_id": "unified_thread_1"}}

# First interaction - should use semantic + long-term + episodic memory
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="What investment strategy would you recommend given my profile?")],
        "user_id": "user_alex",  # Alex has moderate risk tolerance and ESG preferences!
        "feedback": ""
    },
    config
)

print("User: What investment strategy would you recommend given my profile?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\n" + "="*60)
print("Memory types used:")
print("  Long-term: Knows Alex's risk tolerance, portfolio, and ESG preferences")
print("  Semantic: Retrieved investment knowledge from Stone Ridge letter")
print("  Episodic: May use similar advisory episode as reference")
print("  Procedural: Following current instructions")
print("  Short-term: Will remember this in follow-up questions")

User: What investment strategy would you recommend given my profile?

Assistant: To provide a tailored investment strategy, I need to understand your risk tolerance and investment goals. Here are a few clarifying questions:

1. **Risk Tolerance**: How comfortable are you with market fluctuations? Would you prefer a more conservative approach with lower risk, or are you open to higher risk for potentially higher returns?

2. **Investment Goals**: What are your primary investment objectives? Are you looking for long-term growth, income generation, or capital preservation?

3. **Time Horizon**: What is your investment time frame? Are you investing for a short-term goal (1-3 years), medium-term (3-10 years), or long-term (10+ years)?

Once I have this information, I can provide you with actionable insights based on evidence-based investment strategies.

Memory types used:
  Long-term: Knows Alex's risk tolerance, portfolio, and ESG preferences
  Semantic: Retrieved investment knowledge fro

In [36]:
# Follow-up question (tests short-term memory)
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="Can you tell me more about the alternative investments you mentioned?")],
        "user_id": "user_alex",
        "feedback": ""
    },
    config  # Same thread
)

print("User: Can you tell me more about the alternative investments you mentioned?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The agent remembers the context from the previous message!")

User: Can you tell me more about the alternative investments you mentioned?

Assistant: Certainly! Alternative investments refer to asset classes that fall outside of traditional investments like stocks, bonds, and cash. They can provide diversification and potentially enhance returns, but they also come with unique risks. Here are some common types of alternative investments:

1. **Real Estate**: Investing in physical properties or real estate investment trusts (REITs) can provide income through rent and potential appreciation in property value. However, real estate markets can be volatile and require significant capital.

2. **Private Equity**: This involves investing in private companies or buyouts of public companies. While private equity can offer high returns, it typically requires a longer investment horizon and is less liquid than public equities.

3. **Hedge Funds**: These funds employ various strategies to generate returns, including long/short equity, arbitrage, and global m

---
## ❓ Question #3:

How would you decide what constitutes a **"successful" investment advisory interaction** worth storing as an episode? What metadata should you store alongside the episode?

Consider:
- Explicit feedback (thumbs up) vs implicit signals
- User engagement (did they ask follow-up questions?)
- Objective outcomes vs subjective satisfaction
- Privacy implications of storing interaction data

##### Answer:
A "successful" interaction should be identified through a combination of **explicit and implicit signals**, since relying on either alone is insufficient. Explicit feedback (thumbs up/down, star ratings) is the strongest signal but rare—most users don't bother. Implicit signals fill the gap: follow-up questions suggest the user found the advice useful enough to explore further, while abrupt topic changes or session abandonment suggest the opposite. A simple scoring heuristic could combine these: `explicit_positive + follow_up_count > 1 + no_negative_feedback` → store as episode.

**Metadata to store alongside each episode:**
- **Situation/context**: user profile snapshot at the time (risk tolerance, horizon, portfolio size)
- **Query category**: diversification, risk management, alternatives, etc.
- **Response structure**: what format worked (bullet list, numbered steps, pros/cons)
- **Engagement metrics**: number of follow-ups, session duration, whether the user acted on advice
- **Timestamp and version**: when the episode occurred and which procedural instructions were active
- **Anonymized user segment**: (e.g., "conservative retiree") rather than PII, to enable retrieval without privacy risk

**Privacy consideration**: episodes should be scrubbed of personally identifiable information before storage. Store the *pattern* (situation → response style → outcome), not the raw conversation with names, account numbers, or portfolio specifics. This lets episodic memory improve the agent without creating a compliance liability.

## ❓ Question #4:

For a **production investment advisory assistant**, which memory types need persistent storage (PostgreSQL) vs in-memory? How would you handle memory across multiple agent instances (e.g., Market Outlook Agent, Strategy Agent, Risk Management Agent)?

Consider:
- Which memories are user-specific vs shared?
- Consistency requirements across agents
- Memory expiration and cleanup policies
- Namespace strategy for multi-agent systems

##### Answer:
Everything but short-term memory requires persistent storage like Postgres. It's acceptable for short-term memory to be stored in-memory per user session because presumably only the current user's session would need to access to that info and there is less harm to the chat experience if only the current session's state gets dropped or overwritten. Every other state type (procedural, semantic, episodic, long-term) would need persisten storage in a shared DB layer because that type of memory needs to be persisted between multiple sessions for different users and that type of memory often takes up more storage space, potentially more space than can fit in a single machine's RAM. 

Memory across multiple agent instances can be stored in agent-specific blocks in a properly namespaced environment. Memory blocks can be organized in persisten strorage by some combination of the agent_id, session_id and memory type (episodic, procedural, etc.). When handling memory across multiple agent instances, you could break up the blocks by information type (market info, strategy info, risk mgmt info, etc.), time the specific block was produced (today, last week, last month, ...) and which agent is responsible for synthesizing that block of info. 

---
## 🏗️ Activity #2: Investment Memory Dashboard

Build an investment tracking system that:
1. Tracks investment metrics over time (portfolio value, risk score, allocation drift)
2. Uses semantic memory to find relevant investment advice
3. Uses episodic memory to recall what advisory approaches worked before
4. Uses procedural memory to adapt advice style
5. Provides a synthesized "investment summary"

### Requirements:
- Store at least 3 investment metrics per user
- Track metrics over multiple "days" (simulated)
- Agent should reference historical data in responses
- Generate a personalized investment summary

In [38]:
### YOUR CODE HERE ###

# Step 1: Define investment metrics schema and storage functions
def log_investment_metric(store, user_id: str, date: str, metric_type: str, value: float, notes: str = ""):
    """Log an investment metric for a user."""
    namespace = (user_id, "metrics", metric_type)
    store.put(namespace, date, {
        "value": value,
        "date": date,
        "metric_type": metric_type,
        "notes": notes,
        "text": f"{metric_type} on {date}: {value}. {notes}"  # for semantic search
    })


def get_investment_history(store, user_id: str, metric_type: str = None, days: int = 7) -> list:
    """Get investment history for a user, optionally filtered by metric type."""
    results = []
    metric_types = [metric_type] if metric_type else ["portfolio_value", "risk_score", "allocation_drift"]
    for mt in metric_types:
        namespace = (user_id, "metrics", mt)
        items = list(store.search(namespace))
        results.extend([{"key": item.key, **item.value} for item in items])
    # Sort by date
    results.sort(key=lambda x: x.get("date", ""))
    return results[-days:] if len(results) > days else results


# Step 2: Create sample investment data for a user (simulate a week)
sample_metrics = [
    # (date, metric_type, value, notes)
    ("2026-02-10", "portfolio_value", 512000, "Slight uptick from tech rally"),
    ("2026-02-10", "risk_score", 0.42, "Moderate risk band"),
    ("2026-02-10", "allocation_drift", 2.1, "Within tolerance"),
    ("2026-02-11", "portfolio_value", 508500, "Minor pullback in equities"),
    ("2026-02-11", "risk_score", 0.45, "Slightly elevated"),
    ("2026-02-11", "allocation_drift", 3.0, "Approaching rebalance threshold"),
    ("2026-02-12", "portfolio_value", 515200, "Strong recovery day"),
    ("2026-02-12", "risk_score", 0.40, "Back to baseline"),
    ("2026-02-12", "allocation_drift", 2.8, "Stable"),
    ("2026-02-13", "portfolio_value", 510800, "Flat with slight decline"),
    ("2026-02-13", "risk_score", 0.44, "Uptick in volatility"),
    ("2026-02-13", "allocation_drift", 3.5, "Rebalance recommended"),
    ("2026-02-14", "portfolio_value", 518000, "Friday rally, portfolio at weekly high"),
    ("2026-02-14", "risk_score", 0.38, "Risk declining"),
    ("2026-02-14", "allocation_drift", 4.1, "Drift above threshold — rebalance needed"),
]

for date, metric_type, value, notes in sample_metrics:
    log_investment_metric(semantic_store, "user_alex", date, metric_type, value, notes)

print(f"Logged {len(sample_metrics)} metrics for user_alex")

# Verify
history = get_investment_history(semantic_store, "user_alex")
print(f"\nRetrieved {len(history)} records:")
for h in history:
    print(f"  {h['date']} | {h['metric_type']:20s} | {h['value']:>10} | {h['notes']}")


# Step 3: Build an investment dashboard agent (with feedback node)
class DashboardState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str
    feedback: str


def dashboard_agent(state: DashboardState, config: RunnableConfig, *, store: BaseStore):
    """Investment dashboard agent that synthesizes metrics, knowledge, and episodes."""
    user_id = state["user_id"]
    user_message = state["messages"][-1].content

    # Retrieve investment history
    history = get_investment_history(store, user_id)
    if history:
        history_text = "\n".join([
            f"- {h['date']} | {h['metric_type']}: {h['value']} — {h['notes']}"
            for h in history
        ])
    else:
        history_text = "No investment history available."

    # Retrieve user profile (long-term memory)
    profile = get_investment_profile(store, user_id)
    profile_text = "\n".join([f"- {k}: {v}" for k, v in profile.items()]) if profile else "No profile stored."

    # Semantic search for relevant investment advice
    knowledge_results = store.search(("investment", "knowledge"), query=user_message, limit=2)
    knowledge_text = "\n".join([f"- {r.value['text'][:200]}" for r in knowledge_results]) if knowledge_results else "No relevant knowledge found."

    # Episodic memory — find similar past advisory interactions
    episode_results = store.search(("agent", "episodes"), query=user_message, limit=1)
    if episode_results:
        ep = episode_results[0].value
        episode_text = f"A similar past interaction worked well:\nUser asked: {ep.get('input', 'N/A')}\nWhat resonated: {ep.get('feedback', 'N/A')}"
    else:
        episode_text = "No similar past interactions found."

    # Procedural memory — get current instructions
    instructions_item = store.get(("agent", "instructions"), "investment_assistant")
    base_instructions = instructions_item.value["instructions"] if instructions_item else "You are a helpful investment dashboard assistant."

    system_message = f"""{base_instructions}

You are providing an investment dashboard summary. Use ALL of the following context:

=== CLIENT PROFILE ===
{profile_text}

=== INVESTMENT METRICS (RECENT HISTORY) ===
{history_text}

=== RELEVANT INVESTMENT KNOWLEDGE ===
{knowledge_text}

=== LEARNING FROM EXPERIENCE ===
{episode_text}

Analyze trends in the metrics, flag any concerns (drift, risk spikes), and give actionable recommendations.
Reference specific numbers and dates from the history."""

    messages = [SystemMessage(content=system_message)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


def dashboard_feedback_node(state: DashboardState, config: RunnableConfig, *, store: BaseStore):
    """Update procedural memory based on dashboard feedback."""
    feedback = state.get("feedback", "")
    if not feedback:
        return {}

    item = store.get(("agent", "instructions"), "investment_assistant")
    if item is None:
        return {}

    current = item.value
    reflection_prompt = f"""Update these investment dashboard instructions based on user feedback:

Current instructions:
{current['instructions']}

User feedback:
{feedback}

Output only the updated instructions."""

    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    new_version = current["version"] + 1
    store.put(
        ("agent", "instructions"),
        "investment_assistant",
        {"instructions": response.content, "version": new_version}
    )
    print(f"  [Procedural memory updated to v{new_version}]")
    return {}


def dashboard_route(state: DashboardState) -> str:
    """Route to feedback node only when feedback is provided."""
    return "feedback" if state.get("feedback") else "end"


# Build the dashboard graph with feedback loop
dashboard_builder = StateGraph(DashboardState)
dashboard_builder.add_node("dashboard", dashboard_agent)
dashboard_builder.add_node("feedback", dashboard_feedback_node)
dashboard_builder.add_edge(START, "dashboard")
dashboard_builder.add_conditional_edges("dashboard", dashboard_route, {"feedback": "feedback", "end": END})
dashboard_builder.add_edge("feedback", END)

dashboard_graph = dashboard_builder.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)
print("\nDashboard agent compiled (with feedback node)")


# Step 4: Test the dashboard with feedback loop
# --- TEST 1: Initial query (no feedback) ---
print("\n" + "=" * 60)
print("TEST 1: Initial weekly summary (no feedback)")
print("=" * 60)
thread_cfg = {"configurable": {"thread_id": "dashboard_feedback_test"}}

response = dashboard_graph.invoke(
    {
        "messages": [HumanMessage(content="Give me a summary of my portfolio performance this week. Highlight any risks or actions I should take.")],
        "user_id": "user_alex",
        "feedback": ""
    },
    thread_cfg
)
print(f"Assistant: {response['messages'][-1].content}")

# Snapshot the current procedural instructions version
pre_feedback_item = semantic_store.get(("agent", "instructions"), "investment_assistant")
print(f"\nProcedural instructions version BEFORE feedback: v{pre_feedback_item.value['version']}")

# --- TEST 2: Same thread, provide feedback to refine style ---
print("\n" + "=" * 60)
print("TEST 2: Provide feedback — 'Always frame advice in terms of my 20-year horizon'")
print("=" * 60)

response = dashboard_graph.invoke(
    {
        "messages": [HumanMessage(content="I appreciate the summary! Remember, I'm investing for the next 20 years, so always frame your advice around that long horizon.")],
        "user_id": "user_alex",
        "feedback": "Always frame investment advice in terms of the client's long-term horizon (20 years). Emphasize compounding, time-in-market, and long-run trends over short-term volatility."
    },
    thread_cfg
)
print(f"Assistant: {response['messages'][-1].content}")

# Confirm procedural memory was updated
post_feedback_item = semantic_store.get(("agent", "instructions"), "investment_assistant")
print(f"\nProcedural instructions version AFTER feedback: v{post_feedback_item.value['version']}")

# --- TEST 3: Follow-up in same thread — verify updated instructions take effect ---
print("\n" + "=" * 60)
print("TEST 3: Follow-up — does the agent now frame advice around the 20-year horizon?")
print("=" * 60)

response = dashboard_graph.invoke(
    {
        "messages": [HumanMessage(content="Given the allocation drift this week, should I rebalance now or wait?")],
        "user_id": "user_alex",
        "feedback": ""
    },
    thread_cfg
)
print(f"Assistant: {response['messages'][-1].content}")
print("\nNotice: The response should now emphasize long-term horizon framing from the updated procedural memory.")

Logged 15 metrics for user_alex

Retrieved 7 records:
  2026-02-12 | allocation_drift     |        2.8 | Stable
  2026-02-13 | portfolio_value      |     510800 | Flat with slight decline
  2026-02-13 | risk_score           |       0.44 | Uptick in volatility
  2026-02-13 | allocation_drift     |        3.5 | Rebalance recommended
  2026-02-14 | portfolio_value      |     518000 | Friday rally, portfolio at weekly high
  2026-02-14 | risk_score           |       0.38 | Risk declining
  2026-02-14 | allocation_drift     |        4.1 | Drift above threshold — rebalance needed

Dashboard agent compiled (with feedback node)

TEST 1: Initial weekly summary (no feedback)
Assistant: Here's a summary of your portfolio performance for the week, along with identified risks and recommended actions:

### Portfolio Performance Summary:
- **February 12, 2026**: Portfolio value was stable with an allocation drift of **2.8%**.
- **February 13, 2026**: Portfolio value slightly declined to **$510,800**.

---
## Summary

In this module, we explored the **5 memory types** from the CoALA framework:

| Memory Type | LangGraph Component | Scope | Investment Use Case |
|-------------|---------------------|-------|-------------------|
| **Short-term** | `MemorySaver` + `thread_id` | Within thread | Current consultation |
| **Long-term** | `InMemoryStore` + namespaces | Across threads | User profile, goals, constraints |
| **Semantic** | Store + embeddings + `search()` | Across threads | Investment knowledge retrieval |
| **Episodic** | Store + few-shot examples | Across threads | Past successful interactions |
| **Procedural** | Store + self-reflection | Across threads | Self-improving instructions |

### Key Takeaways:

1. **Memory transforms chatbots into advisors** - Persistence enables personalization
2. **Different memory types serve different purposes** - Choose based on your use case
3. **Context management is critical** - Trim and summarize to stay within limits
4. **Episodic memory enables learning** - Show, don't just tell
5. **Procedural memory enables adaptation** - Agents can improve themselves

### Production Considerations:

- Use `PostgresSaver` instead of `MemorySaver` for persistent checkpoints
- Use `PostgresStore` instead of `InMemoryStore` for persistent long-term memory
- Consider TTL (Time-to-Live) policies for automatic memory cleanup
- Implement proper access controls for user data
- Ensure compliance with financial regulations for investment advisory data

### Further Reading:

- [LangGraph Memory Documentation](https://langchain-ai.github.io/langgraph/concepts/memory/)
- [CoALA Paper](https://arxiv.org/abs/2309.02427) - Cognitive Architectures for Language Agents
- [LangGraph Platform](https://docs.langchain.com/langgraph-platform/) - Managed infrastructure for production